In [ ]:
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 12.7 MB/s 


In [ ]:
import os
import pandas as pd
import numpy as np
import pydicom
from google.colab import drive
import zipfile
import re
import cv2
import json 

## Download dataset for prediction

##### From Kaggle competition:
https://www.kaggle.com/competitions/unifesp-x-ray-body-part-classifier

##### Usually the DICOM data is stored in many subfolders, you can load them all together in the parent folder using Total Commander or a terminal query: 
`find /Users/tonyd/Desktop/xray/train/ -mindepth 2 -type f -exec mv -i '{}' /Users/tonyd/Desktop/xray/train/ ';'`

### Download images to Colab as zip (it's working faster) from Google Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file='/content/drive/My Drive/test.zip'
z=zipfile.ZipFile(zip_file,'r')
z.extractall()

### Download CSV file with file names

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

### Get filenames list

In [ ]:
file_names = list(test_df['SOPInstanceUID'])

In [ ]:
folder_path = r"/content/"
ds = pydicom.dcmread(file_path)

### Create features (X) from images and targets (y) from file

In [ ]:
X_temp=[]

In [ ]:
for el in file_names:
  file_name = el + '-c.dcm'                                                # get filename with img
  file_path = os.path.join(folder_path,file_name)                          # get img adress
  ds = pydicom.dcmread(file_path)                                          # transform from DICOM
  data = np.array(ds.pixel_array)                                          # transform to numpy array
  image = cv2.resize(data, (84, 84))                                       # resize
  image=image/255                                                          # change grayscale to 256 format
  X_temp.append(image)                                                     # add features to 'X'


/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:239: UserWarning: The (0028,0101) 'Bits Stored' value (15-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({bits_stored}-bit) "
/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:239: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({bits_stored}-bit) "
/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:239: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({bits_stored}-bit) "


In [ ]:
X=np.array(X_temp)                                        # make a np.array 
X = X.reshape(X.shape[0], (X.shape[1] * X.shape[2]))      # reshape 3d array to 2d for sving to CSV file

In [ ]:
pd.DataFrame(X).to_csv("comp.csv", header = False , index = False)              # save features (X)